In [4]:
import folium
import pandas as pd
import numpy as np

data = pd.read_csv('Floating_population_2010.csv')
location = pd.read_csv('서울시 행정구역 시군구 정보 (좌표계_ WGS1984).csv', encoding='cp949')

In [5]:
location = location.drop(['순번', '시군구코드', '시군구명_영문', 'ESRI_PK'], axis = 1)
data = data.drop(['연령대(10세단위)', '성별', '시'], axis = 1)

In [6]:
# data = data[(data['군구']=='강남구')&(data['일자'] <= 20201001)&((data['시간(1시간단위)']>=7)&(data['시간(1시간단위)']<=10))]

# data = data[((data['군구']=='강남구')|(data['군구']=='송파구'))&(data['일자'] <= 20201001)]
data = data[data['일자'] <= 20201001]

In [7]:
data = data.drop(['일자'], axis = 1)
data = data.groupby(['군구', '시간(1시간단위)']).sum().reset_index()

In [8]:
location = location.rename({"시군구명_한글":"군구"}, axis = "columns")

In [9]:
df = pd.merge(data, location, on = "군구")

In [10]:
## 시간대별 남성, 여성의 유동인구 수 0~100스케일링(상대적)

value = df['유동인구수'].values
minimum = min(value)
maximum = max(value)
df['유동인구수'] = df['유동인구수'].transform(func = lambda x : 100*(x - minimum)/(maximum-minimum))
# df['유동인구수'] = df['유동인구수'].transform(func = lambda x : int(str(x)[:3]))
df['유동인구수'] = np.ceil(df['유동인구수']).astype(int)

In [15]:
date = list(df['시간(1시간단위)'].unique())

In [16]:
matrix = []
for i in date : ## 24
    a = []
    temp = df[df['시간(1시간단위)']==i]
    
    for lat, lon, value in zip(temp['위도'], temp['경도'], temp['유동인구수']) :
        [a.append([lat+np.random.normal(0,0.0008), lon+np.random.normal(0,0.0008)]) for i in range(value)]
        
        matrix.append(a)

In [17]:
print('전체 리스트 수 : ', len(matrix))
print('첫번째 리스트 안의 데이터 개수 : ', len(matrix[5]))

전체 리스트 수 :  600
첫번째 리스트 안의 데이터 개수 :  1356


In [18]:
new_mat = []
for idx,i in enumerate(matrix) :
    if idx%25==0 :
        new_mat.append(i)
    else :
        pass

In [19]:
def change_date(x) :
    x = str(x)
    return x +'시'

time_ind = list(map(lambda i : change_date(i), date))
print(time_ind)

['0시', '1시', '2시', '3시', '4시', '5시', '6시', '7시', '8시', '9시', '10시', '11시', '12시', '13시', '14시', '15시', '16시', '17시', '18시', '19시', '20시', '21시', '22시', '23시']


In [20]:
import folium.plugins as plugins

center = [37.5579452,126.9941904]

m = folium.Map(location=center, zoom_start=11)
plugins.BeautifyIcon
plugins.HeatMapWithTime(new_mat, index=time_ind, auto_play=True).add_to(m)
m


In [21]:
m.save('C:/Users/dylee02/10월01일_시간대별_서울유동인구_시각화(상대적).html')